In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18400565
paper_name = 'lis_romesberg_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, sep='\t')

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 268 x 5


In [10]:
original_data.head()

,0,1,2,3,4
0,1,YJR139C,HOM6,7.250000e-09,9.000000e-09
1,2,YDR514C,NaN,2.260000e-08,3.600000e-08
2,3,YDR300C,PRO1,1.290000e-07,8.010000e-07
3,4,YML010W-A,NaN,4.180000e-07,1.930000e-08
4,5,YGL033W,HOP2,4.300000e-07,2.400000e-08


In [11]:
original_data['orf'] = original_data[1].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

               0   1    2        3             4 orf
index_input                                         
201          202  WT  NaN  0.00012  2.050000e-07  WT


In [15]:
original_data.set_index('orf', inplace=True)
original_data.drop(columns=[0,1,2], inplace=True)

In [16]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [17]:
# Normalize by WT
original_data = original_data.div(original_data.loc['WT',:], axis=1)

In [19]:
original_data.drop(index='WT', inplace=True)

In [20]:
original_data = original_data.groupby(original_data.index).mean()

In [21]:
original_data.shape

(264, 2)

# Prepare the final dataset

In [22]:
data = original_data.copy()

In [23]:
dataset_ids = [1188,11857]
datasets = datasets.reindex(index=dataset_ids)

In [24]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [25]:
data.head()

dataset_id,1188,11857
data_type,value,value
orf,,
YAL010C,0.376667,0.682927
YAL035W,1.241667,1.780488
YAL047C,0.012833,0.269756
YAL054C,0.875000,2.209756
YAL056C-A,1.250000,0.985366


## Subset to the genes currently in SGD

In [26]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [27]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,1188,11857
,data_type,value,value
gene_id,orf,,
8,YAL010C,0.376667,0.682927
33,YAL035W,1.241667,1.780488
45,YAL047C,0.012833,0.269756
50,YAL054C,0.875000,2.209756
1866,YAL056C-A,1.250000,0.985366


# Normalize

In [28]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [29]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [30]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,1188,11857,1188,11857
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
8,YAL010C,0.376667,0.682927,1.724363,0.863709
33,YAL035W,1.241667,1.780488,5.684295,2.251812
45,YAL047C,0.012833,0.269756,0.058750,0.341165
50,YAL054C,0.875000,2.209756,4.005711,2.794714
1866,YAL056C-A,1.250000,0.985366,5.722445,1.246208


# Print out

In [31]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
from IO.save_data_to_db3 import *

In [33]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 18400565...
Inserting the new data...


100%|██████████| 2/2 [00:01<00:00,  1.95it/s]

Updating the data_modified_on field...
